In [1]:
pip install librosa

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 73.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 41.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scipy==1.9.3


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 985.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 49.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 50.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.0
    Uninstalling scipy-1.13.0:
      Successfully uninstalled scipy-1.13.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import scipy

In [3]:
folder = '../../mnt/disks/songsnap/audio'
all_files = os.listdir(folder)

In [4]:
def generateSoundFeatures(file_name, folder):
    if file_name.endswith('.wav'):
        audio_path = os.path.join(folder, file_name)

        base_name = os.path.splitext(file_name)[0]

        y, sr = librosa.load(audio_path, duration=30.0)
    
        # Mel-frequency Cepstral coefficients
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
        # Chroma features
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
        # Spectral centroid
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        # Spectral bandwith
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        # Tempo
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        # Zero crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))
    
        return np.hstack([base_name, mfccs, chroma, spectral_centroid, spectral_bandwidth, 
                              spectral_contrast, spectral_rolloff, tempo, zero_crossing_rate])
        
        


In [ ]:
idx = 2
for file_name in tqdm(all_files[1000:]): 
    features = []
    result = generateSoundFeatures(file_name, folder)
    features.append(result)
    
    mfcc_headers = [f'mfcc_{i}' for i in range(13)]
    chroma_headers = [f'chroma_{i}' for i in range(12)]
    
    columns = ['file']
    columns.extend(mfcc_headers)
    columns.extend(chroma_headers)
    columns.extend(['spectral_centroid', 'spectral_bandwidth', 'spectral_contrast', 'spectral_rolloff',
                  'tempo', 'zero_crossing_rate'])
    
    feature_df = pd.DataFrame(features, columns=columns)
    path = '../../mnt/disks/songsnap/sound/sound_features_' + str(idx) +'.csv'
    
    feature_df.to_csv(path, index=False)
    idx = idx+1

  7%|▋         | 1925/27730 [25:02<5:36:54,  1.28it/s]

In [1]:
import pandas as pd

dfs = []

for i in range(1, 1949):
    filename = f"../../mnt/disks/songsnap/sound/sound_features_{i}.csv"
    
    # Read the CSV file into a pandas dataframe
    df = pd.read_csv(filename)
    
    # Append the dataframe to the list
    dfs.append(df)

# Concatenate all the dataframes in the list along the rows
composite_df = pd.concat(dfs, ignore_index=True)

In [2]:
composite_df.shape

(2947, 32)

In [3]:
composite_df.to_csv("../../mnt/disks/songsnap/sound/composite_sound_features.csv", index=False)